下载 CBDB Sqlite 数据库保存到 'Introduction-to-Digital-Humanities/SNA/data' 目录下 
下载链接：https://projects.iq.harvard.edu/cbdb/download-cbdb-standalone-database
中国人物传记数据库是一个可免费访问的关系数据库，其中包含截至 2024 年 8 月的约 641,568 人的传记信息，目前主要涵盖 7 世纪至 19 世纪的人物。该数据库有在线和离线版本，可用于统计、社交网络和空间分析，也可用作传记参考。

In [6]:
import sqlite3

# 连接到 SQLite 数据库文件
db_path = "/Volumes/My Passport/GitHub/Introduction-to-Digital-Humanities/SNA/data/CBDB.db"  # 数据库文件的路径
conn = sqlite3.connect(db_path)  # 创建与数据库的连接

# 创建一个 cursor 对象，用于执行 SQL 查询
cursor = conn.cursor()

# 查询数据库中的所有表
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()  # 获取表的名称

# 遍历每个表，输出该表的列和数据类型
for table in tables:
    table_name = table[0]  # 表名
    print(f"\nTable: {table_name}")
    
    # 使用 PRAGMA 语句获取表的列信息
    cursor.execute(f"PRAGMA table_info({table_name});")
    columns = cursor.fetchall()  # 获取列的信息
    
    # 输出每个列的名称和数据类型
    for column in columns:
        print(f"Column: {column[1]}, Data Type: {column[2]}")

# 关闭数据库连接
conn.close()


Table: ADDR_BELONGS_DATA
Column: c_addr_id, Data Type: INTEGER
Column: c_belongs_to, Data Type: INTEGER
Column: c_firstyear, Data Type: INTEGER
Column: c_lastyear, Data Type: INTEGER
Column: c_source, Data Type: INTEGER
Column: c_pages, Data Type: CHAR(255)
Column: c_secondary_source_author, Data Type: CHAR(255)
Column: c_notes, Data Type: CHAR(255)

Table: ADDR_CODES
Column: c_addr_id, Data Type: INTEGER
Column: c_name, Data Type: CHAR(255)
Column: c_name_chn, Data Type: CHAR(255)
Column: c_firstyear, Data Type: INTEGER
Column: c_lastyear, Data Type: INTEGER
Column: c_admin_type, Data Type: CHAR(255)
Column: x_coord, Data Type: FLOAT
Column: y_coord, Data Type: FLOAT
Column: CHGIS_PT_ID, Data Type: INTEGER
Column: c_notes, Data Type: CHAR
Column: c_alt_names, Data Type: CHAR(255)

Table: ADDR_PLACE_DATA
Column: c_addr_id, Data Type: INTEGER
Column: c_place_id, Data Type: INTEGER
Column: c_firstyear, Data Type: INTEGER
Column: c_lastyear, Data Type: INTEGER

Table: ADDR_XY
Column: c_a

In [12]:
import sqlite3

# 连接到源数据库文件
source_db_path = "/Volumes/My Passport/GitHub/Introduction-to-Digital-Humanities/SNA/data/CBDB.db"  # 源数据库文件的路径
conn_source = sqlite3.connect(source_db_path)  # 创建与源数据库的连接
cursor_source = conn_source.cursor()

# 创建或连接到目标数据库文件
target_db_path = "CBDB_examples.db"  # 目标数据库文件的路径
conn_target = sqlite3.connect(target_db_path)  # 创建与目标数据库的连接
cursor_target = conn_target.cursor()

# 查询源数据库中的所有表
cursor_source.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor_source.fetchall()  # 获取表的名称

# 数据类型映射：根据需要修改为 SQLite 支持的类型
def map_data_type(data_type):
    data_type = data_type.upper()
    if "INT" in data_type:
        return "INTEGER"
    elif "CHAR" in data_type or "TEXT" in data_type or "CLOB" in data_type:
        return "TEXT"
    elif "BLOB" in data_type:
        return "BLOB"
    elif "REAL" in data_type or "FLOA" in data_type or "DOUB" in data_type:
        return "REAL"
    else:
        return "TEXT"  # 默认转换为 TEXT 类型

# 遍历每个表，提取前100条数据并保存到目标数据库
for table in tables:
    table_name = table[0]  # 表名
    print(f"\nProcessing table: {table_name}")
    
    # 使用 PRAGMA 语句获取表的列信息
    cursor_source.execute(f"PRAGMA table_info({table_name});")
    columns = cursor_source.fetchall()  # 获取列的信息
    
    # 创建表结构的 SQL 语句
    column_definitions = ", ".join([f"{col[1]} {map_data_type(col[2])}" for col in columns])  # 列名和数据类型
    create_table_sql = f"CREATE TABLE IF NOT EXISTS {table_name} ({column_definitions});"
    try:
        cursor_target.execute(create_table_sql)  # 在目标数据库中创建表
        conn_target.commit()
    except sqlite3.OperationalError as e:
        print(f"Error creating table {table_name}: {e}")
        continue

    # 查询前100条数据
    cursor_source.execute(f"SELECT * FROM {table_name} LIMIT 100;")
    rows = cursor_source.fetchall()  # 获取前100条数据
    
    # 准备插入数据的 SQL 语句
    placeholders = ", ".join(["?"] * len(columns))  # 占位符，用于插入数据
    insert_sql = f"INSERT INTO {table_name} VALUES ({placeholders});"
    
    # 将数据插入到目标数据库
    try:
        cursor_target.executemany(insert_sql, rows)
        conn_target.commit()  # 提交插入操作
        print(f"Inserted {len(rows)} rows into {table_name}.")
    except sqlite3.OperationalError as e:
        print(f"Error inserting data into {table_name}: {e}")

# 关闭数据库连接
conn_source.close()
conn_target.close()

print("Data extraction and insertion complete.")


Processing table: ADDR_BELONGS_DATA
Inserted 100 rows into ADDR_BELONGS_DATA.

Processing table: ADDR_CODES
Inserted 100 rows into ADDR_CODES.

Processing table: ADDR_PLACE_DATA
Inserted 100 rows into ADDR_PLACE_DATA.

Processing table: ADDR_XY
Inserted 100 rows into ADDR_XY.

Processing table: ADDRESSES
Inserted 100 rows into ADDRESSES.

Processing table: ALTNAME_CODES
Inserted 21 rows into ALTNAME_CODES.

Processing table: ALTNAME_DATA
Inserted 100 rows into ALTNAME_DATA.

Processing table: APPOINTMENT_TYPE_CODES
Error creating table APPOINTMENT_TYPE_CODES: near "INTEGER": syntax error

Processing table: ASSOC_CODE_TYPE_REL
Inserted 100 rows into ASSOC_CODE_TYPE_REL.

Processing table: ASSOC_CODES
Inserted 100 rows into ASSOC_CODES.

Processing table: ASSOC_DATA
Inserted 100 rows into ASSOC_DATA.

Processing table: ASSOC_TYPES
Inserted 45 rows into ASSOC_TYPES.

Processing table: ASSUME_OFFICE_CODES
Inserted 6 rows into ASSUME_OFFICE_CODES.

Processing table: BIOG_ADDR_CODES
Inserte

In [19]:
import sqlite3
import pandas as pd

# 用户自定义c_assoc_code集合，例如：[22, 23]
custom_assoc_codes = [22, 23]  # 你可以根据需求修改此集合

# 连接到数据库
db_path = '/Volumes/My Passport/GitHub/Introduction-to-Digital-Humanities/SNA/data/CBDB.db'  # 替换为你的数据库路径
conn = sqlite3.connect(db_path)

# 编写SQL查询，将c_personid和c_assoc_id替换为c_name_chn，并加入c_assoc_desc_chn
query = f"""
SELECT 
    a.c_assoc_code,
    ac.c_assoc_desc_chn,
    p1.c_name_chn AS person_name,
    d1.c_dynasty_chn AS person_dynasty_chn,
    p2.c_name_chn AS assoc_name,
    d2.c_dynasty_chn AS assoc_dynasty_chn
FROM ASSOC_DATA a
JOIN BIOG_MAIN p1 ON a.c_personid = p1.c_personid
JOIN BIOG_MAIN p2 ON a.c_assoc_id = p2.c_personid
JOIN DYNASTIES d1 ON p1.c_dy = d1.c_dy
JOIN DYNASTIES d2 ON p2.c_dy = d2.c_dy
JOIN ASSOC_CODES ac ON a.c_assoc_code = ac.c_assoc_code
WHERE a.c_assoc_code IN ({', '.join(map(str, custom_assoc_codes))})
AND (d1.c_dynasty_chn = '清' OR d2.c_dynasty_chn = '清')
"""

# 使用pandas执行SQL查询并将结果加载为DataFrame
df = pd.read_sql_query(query, conn)

# 关闭数据库连接
conn.close()

# 保存结果到Excel文件
excel_file_path = 'filtered_and_joined_assoc_data.xlsx'  # 自定义文件名和路径
df.to_excel(excel_file_path, index=False)

# 显示保存文件的路径
print(f"数据已保存到 {excel_file_path}")

数据已保存到 filtered_and_joined_assoc_data.xlsx


In [2]:
import sqlite3
import pandas as pd

# 用户自定义c_assoc_code集合，例如：[22, 23]
custom_assoc_codes = [22, 23]  # 你可以根据需求修改此集合

# 连接到数据库
db_path = '/Volumes/My Passport/GitHub/Introduction-to-Digital-Humanities/SNA/data/CBDB.db'  # 替换为你的数据库路径
conn = sqlite3.connect(db_path)


# 编写SQL查询，将c_personid和c_assoc_id替换为c_name_chn，并加入c_assoc_desc_chn
query = f"""
SELECT 
    a.c_assoc_code,
    ac.c_assoc_desc_chn,
    p1.c_name_chn AS person_name,
    d1.c_dynasty_chn AS person_dynasty_chn,
    p2.c_name_chn AS assoc_name,
    d2.c_dynasty_chn AS assoc_dynasty_chn
FROM ASSOC_DATA a
JOIN BIOG_MAIN p1 ON a.c_personid = p1.c_personid
JOIN BIOG_MAIN p2 ON a.c_assoc_id = p2.c_personid
JOIN DYNASTIES d1 ON p1.c_dy = d1.c_dy
JOIN DYNASTIES d2 ON p2.c_dy = d2.c_dy
JOIN ASSOC_CODES ac ON a.c_assoc_code = ac.c_assoc_code
WHERE a.c_assoc_code IN ({', '.join(map(str, custom_assoc_codes))})
AND (d1.c_dynasty_chn = '清' OR d2.c_dynasty_chn = '清')
"""

# 使用pandas执行SQL查询并将结果加载为DataFrame
df = pd.read_sql_query(query, conn)

# 关闭数据库连接
conn.close()

# 创建节点（去重）并保存为CSV文件
nodes = pd.DataFrame({
    'Id': pd.concat([df['person_name'], df['assoc_name']]).unique(),
    'Label': pd.concat([df['person_name'], df['assoc_name']]).unique()
})
nodes.to_csv('nodes.csv', index=False)

# 创建边并保存为CSV文件
edges = pd.DataFrame({
    'Source': df['person_name'],
    'Target': df['assoc_name'],
    'Type': 'Directed',  # 可视化为无向图
    'Label': df['c_assoc_desc_chn'],  # 使用关系描述
    'Weight': 1  # 可以自定义权重
})
edges.to_csv('edges.csv', index=False)

# 显示保存文件的路径
print("节点数据已保存到 nodes.csv")
print("边数据已保存到 edges.csv")

节点数据已保存到 nodes.csv
边数据已保存到 edges.csv
